## PyCaret 설치

In [ ]:
!pip install pycaret

Google Colab 사용자의 경우 다음의 코드를 실행합니다.

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## 필요한 모듈 import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.display.max_columns = None

## 실습을 위한 데이터셋 로드

In [ ]:
from pycaret.datasets import get_data
dataset = get_data('france')

- `InvoiceNo`: 송장번호. 명목, 각 거래에 고유하게 할당된 6자리 정수. 이 코드가 문자 'c'로 시작하면 취소를 나타냅니다.
- `StockCode`: 제품(항목) 코드. 공칭, 각 고유 제품에 고유하게 할당된 5자리 정수입니다.
- `Description`: 제품(항목) 이름
- `Quantity`: 거래당 각 제품(항목)의 수량
- `InvoiceData`: 송장 날짜 및 시간
- `UnitPrice`: 단가. 숫자, 단위당 제품 가격.
- `CustomerID`: 고객 번호입니다. 호칭, 각 고객에게 고유하게 할당된 5자리 정수
- `Country`: 국가 이름. 명목상, 각 고객이 거주하는 국가의 이름

## STEP 1) 셋업 setup

머신러닝 예측 방식에 따라 다음 중 하나를 골라 import 합니다.

In [ ]:
# from pycaret.classification import *  # 분류
# from pycaret.regression import *      # 회귀
# from pycaret.clustering import *      # 군집
# from pycaret.anomaly import *         # 이상 탐지
from pycaret.arules import *            # 연관 규칙

**`setup` 함수**

- `data`: 학습할 데이터셋을 지정합니다.
- `transaction_id`: 트랜섹션 id를 입력합니다. `transaction_id`를 기준으로 아이템을 그룹핑합니다.
- `item_id`: 아이템의 고유 품목(명) 이 되는 컬럼을 지정합니다.
- `session_id`: SEED 값을 지정합니다.

In [ ]:
train.head(1)

In [ ]:
clf = setup(data=dataset, 
            transaction_id='InvoiceNo',
            item_id='Description') 

- `Transactions` : 데이터세트의 고유한 트랜잭션 수입니다. 이 경우 `InvoiceNo`의 고유 개수가 됩니다.

- `Items` : 데이터세트의 고유한 항목 수입니다. 이 경우 `Description`에 지정한 고유 품목수가 됩니다.

- `Ignore Items` : 연관 규칙에서 무시할 항목을 지정합니다. 

In [ ]:
clf = setup(data=dataset, 
            transaction_id='InvoiceNo',
            item_id='Description', 
            ignore_items = ['POSTAGE']
            ) 

## 모델 생성

연관규칙에서 모델을 생성하는 방법은 단순합니다. 필수로 입력해야 하는 파라미터는 없으나 4개의 옵션을 제공하며 옵션은 다음과 같습니다.

- `metric`: 평가지표를 설정합니다. 기본값은 `confidence`입니다. 다른 옵션 값으로는 `support`, `lift`, `leverage`, `conviction` 가 있습니다.
- `threshold`: 평가지표에 대한 최소 임계값을 지정합니다. 기본 값은 **0.5** 입니다.
- `min_support`: `support`에 대한 최소 임계값을 지정합니다. `support는 다음과 같이 계산 됩니다. `트랜섹션 별 아이템 출현 빈도` / `전체 트랜섹션`. 기본 값은 **0.05** 입니다.
- `round`: 반올림할 소수점 자릿수를 지정합니다.

In [ ]:
ar_model = create_model() 

In [ ]:
ar_model.shape

## 결과 해석

아이템 X, Y가 존재한다고 가정합니다.

1. **지지도(support)**:
  - 두 항목 X, Y가 모두 포함하는 거래건수(transaction_id)의 비율을 나타냅니다.
  - (X, Y 모두 포함하는 transaction_id) / (전체 transaction_id)
  - (실사례) frequent item set을 판단하는데 사용
  - (예시) 햇반, 라면, 스팸 묶음 세트 -> 마케팅 프로모션

2. **신뢰도(confidence)**:
  - X를 포함하는 거래중 X, Y를 모두 포함하는 조건부 확률을 말합니다.
  - (X, Y 모두 포함하는 transaction_id) / (항목 X가 포함된 transaction_id)
  - 아이템 집합 간의 **연관성 강도**를 측정할 때 사용
  - 신뢰도 100%의 의미는 X를 구매하는 경우, 무조건 Y를 구매한다고 볼 수 있음
  - 다만, 향상도를 같이 고려해야함

3. **향상도(lift)**:
  - 항목 X가 주어지지 않았을 때 항목 Y의 확률 대비 / 항목 X가 주어졌을 때 항목 Y의 확률 증가 비율을 말합니다.
  - lift가 1보다 큰 경우: 긍정적인 연관관계
  - lift가 1인 경우: X, Y는 독립적인 관계. 연관성이 없음
  - lift가 1보다 작은 경우: 부정적인 연관관계
  - 만약, 두 항목 (X, Y)가 대체제인 경우 같이 구매되는 경우가 없으므로 1보다 작은 값이 발생할 수 있음. (예시) 콜라/사이다.



- `antecedents`: 전례
- `consequents`: 결과

In [ ]:
ar_model[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

In [ ]:
plot_model(ar_model)

In [ ]:
plot_model(ar_model, plot='3d')